In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "itard"
fs_method, fs_ratio = "graphout", 0.5
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


Epoch 1/50:  23%|██▎       | 6/26 [00:00<00:00, 59.19batch/s, loss=0.412]

model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9185 | AUPR: 0.6561 | Acc: 95.41%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9212 | AUPR: 0.6646 | Acc: 95.38%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9238 | AUPR: 0.6737 | Acc: 95.39%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9174 | AUPR: 0.6549 | Acc: 95.36%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9206 | AUPR: 0.6637 | Acc: 95.36%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9234 | AUPR: 0.6717 | Acc: 95.31%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9238 | AUPR: 0.6737 | Acc: 95.39%
Internal test: AUC: 0.9302 | AUPR: 0.7004 | Acc: 95.06%
External test: AUC: 0.9224 | AUPR: 0.6662 | Acc: 95.16%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9182 | AUPR: 0.6355 | Acc: 95.04%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9190 | AUPR: 0.6328 | Acc: 95.02%


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

In [ ]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

In [ ]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

In [ ]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)